In [1]:
!pip3 install minio

In [2]:
%%bash
## create dummy files (sleep 1 to generated random string)
echo `date| md5sum` > /tmp/file01 && sleep 1
echo `date| md5sum` > /tmp/file02 && sleep 1
echo `date| md5sum` > /tmp/file03 && sleep 1
echo `date| md5sum` > /tmp/file04 && sleep 1
echo `date| md5sum` > /tmp/file05
cat /tmp/file0*

d9401376e4974ab6c3e1a607c4a6d1ac -
8d7562f43ff45dc6b6627bcbf843abd9 -
f96884db88be15fd669d2a969cd44dd5 -
6abf4e5fa3ab7fbaf49bd31a29a50ebf -
1323d3e33a1c486fadb7250813161551 -


In [4]:
%%bash
env |grep -i minio

MINIO_SERVICE_SERVICE_PORT_HTTP=9000
MINIO_SERVICE_PORT_9000_TCP_ADDR=10.152.183.226
MINIO_SERVICE_PORT=tcp://10.152.183.226:9000
MINIO_SERVICE_PORT_9000_TCP_PROTO=tcp
MINIO_SERVICE_PORT_9000_TCP_PORT=9000
MINIO_SERVICE_PORT_9000_TCP=tcp://10.152.183.226:9000
MINIO_SERVICE_SERVICE_HOST=10.152.183.226
MINIO_SERVICE_SERVICE_PORT=9000


In [5]:
%%bash
echo minio address:
echo http://$MINIO_SERVICE_SERVICE_HOST:$MINIO_SERVICE_SERVICE_PORT_HTTP

minio address:
http://10.152.183.226:9000


In [6]:
import os
minio_host, minio_port = os.environ["MINIO_SERVICE_SERVICE_HOST"], os.environ["MINIO_SERVICE_SERVICE_PORT_HTTP"]
minio_url= "{}:{}".format(minio_host, minio_port)
print(minio_url)

10.152.183.226:9000


In [7]:
from minio import Minio
from minio.error import S3Error

BUCKET_NAME="datapipeline-021"

config = {
    "endpoint": minio_url,
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False,
    }


# Create a client with the MinIO server playground, its access key
# and secret key.
print ("connecting to minio {}".format(minio_url))
minio_client = Minio(**config)

print("try to find bucket {}".format(BUCKET_NAME))
found = minio_client.bucket_exists(BUCKET_NAME)
print("found", found)
if not found:
    minio_client.make_bucket(BUCKET_NAME)
else:
    print("Bucket '{}' already exists".format(BUCKET_NAME))

# # Upload '/home/user/Photos/asiaphotos.zip' as object name
# # 'asiaphotos-2015.zip' to bucket 'asiatrip'.
# client.fput_object(
# "asiatrip", "asiaphotos-2015.zip", "/home/user/Photos/asiaphotos.zip",
# )
# print(
# "'/home/user/Photos/asiaphotos.zip' is successfully uploaded as "
# "object 'asiaphotos-2015.zip' to bucket 'asiatrip'."
# )


connecting to minio 10.152.183.226:9000
try to find bucket datapipeline-021
found True
Bucket 'datapipeline-021' already exists


In [8]:
import glob
import os
files=glob.glob("/tmp/file0*")
for filepath in files:
    minio_client.fput_object(BUCKET_NAME, os.path.basename(filepath), filepath)

In [9]:

def download_path(filename):
    return "/tmp/minio_download_{}".format(filename)

for item in minio_client.list_objects(BUCKET_NAME,recursive=False):
    print(item.object_name)
    obj=minio_client.fget_object(BUCKET_NAME,item.object_name,download_path(item.object_name))
    print(obj)


file01
file02
file03
file04
file05


In [10]:
%%bash
for f in `ls /tmp/minio_download*`; do
    echo $f
    cat $f
done


/tmp/minio_download_file01
d9401376e4974ab6c3e1a607c4a6d1ac -
/tmp/minio_download_file02
8d7562f43ff45dc6b6627bcbf843abd9 -
/tmp/minio_download_file03
f96884db88be15fd669d2a969cd44dd5 -
/tmp/minio_download_file04
6abf4e5fa3ab7fbaf49bd31a29a50ebf -
/tmp/minio_download_file05
1323d3e33a1c486fadb7250813161551 -
